On-Device Training.


You can use the coremltools_internal APIs to convert training programs from front-end frameworks such as Tensorflow2 Keras and PyTorch to the mlprogram format for on-device training. Use on-device training technologies for Private Federated Learning (PFL) and on-device personalization.

Steps to Convert and Deploy
PyTorch Training Conversion
TensorFlow 2 Keras Training Conversion
Private Federated Learning in Python

Steps to Convert and Deploy

Follow these steps to implement an end-to-end conversion and deployment pipeline for your training programs:

Produce a training Core ML model by converting front-end training mode using coremltools_internal.convert() with training_mode=True (Tensorflow2 Keras and PyTorch are supported). For example:

In [ ]:
import coremltools_internal as ct
#
# Add code to obtain source_model.
#
model = ct.convert(source_model, training_mode=False, convert_to="mlprogram")

Before deploying, test and validate your training program locally using a Python script and the Trainer and Evaluator Python-binding APIs:

coremltools_internal.models.trainer.Trainer: A high-level training API that lets you update the model in a single line.
coremltools_internal.models.evaluator.Evaluator: A low-level API that lets you evaluate each function in the training program. For example, you can initialize the model and compute the forward and backpropogation function and run the training process through your python code.

Deploy the model using Xcode. Use the following command to produce a training model model.mlmodelc and deploy it through Xcode:

In [ ]:
coremlcompiler compile model.mlpackage. 

PyTorch Training Conversion

The following describes the basic model format, using TorchOptimizer, and converting the model.

Basic Model Format

The following is an example of training an MLP model with MSE loss using PyTorch:

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class LinearModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_1 = nn.Linear(2, 1)
        self.linear_2 = nn.Linear(1, 3)

    def forward(self, x):
        x = self.linear_1(x)
        x = self.linear_2(x)
        return x

model = LinearModel()

def criterion(pred, label):
    diff = pred - label
    return torch.mean(diff * diff)

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

x = torch.rand(2, 2)
y = torch.rand(2, 3)

pred = model(x)
loss = criterion(pred, y)
loss.backward()
optimizer.step()

In PyTorch, the loss function can be seperated from the model itself, which is unfortunately not allowed in the coremltools_internal training mode. The Espresso backend, which performs the training program lowering, assumes that the first model output is always the loss function, and the remaining outputs are model predictions.

As the result, in the above example, you need to create a new model, which wraps the loss function along with the predictions. Note that the new model also needs to add the label as an additional input, and the original PyTorch model should be a class member of the new model:

In [ ]:
class ModelToExport(nn.Module):
    def __init__(self):
        super().__init__()

        # The original model is a class member
        self.backend = model

    def forward(self, x, y):
        x = self.backend(x)
        loss = criterion(x, y)

        # Loss must come at first
        return loss, x

model_to_export = ModelToExport()
optimizer = optim.SGD(model_to_export.parameters(), lr=0.001, momentum=0.9)

x = torch.rand(2, 2)
y = torch.rand(2, 3)

loss, pred = model_to_export(x, y)
loss.backward()
optimizer.step()

TorchOptimizer in AdvancedOptions

You also need to pass the Torch optimizer information to the converter using the TorchOptimizer advanced option, which takes the torch model and the torch optimizer as inputs:

In [ ]:
from coremltools_internal import AdvancedOptions, TorchOptimizer

advanced_options = AdvancedOptions()
torch_optimizer = TorchOptimizer(model=model_to_export, optimizer=optimizer)
advanced_options.add_option(torch_optimizer)

Conversion

After completing the above steps, use example inputs to trace the model_to_export using torch.jit.trace(), and pass the traced model to the coremltools_internal.convert() API with training_mode=True:

In [ ]:
import coremltools_internal as ct

example_inputs = [x, y]
traced_model = torch.jit.trace(model_to_export, example_inputs)
inputs = [ct.TensorType(shape=input_.shape) for input_ in example_inputs]
mlmodel = ct.convert(traced_model, inputs=inputs, training_mode=True, advanced_options=advanced_options)